Lets include variables and declaration notebook here

In [9]:
%run variables-ds-pydantic.ipynb

In [10]:
epss_lookup = load_epss_map()

In [51]:
import subprocess
import requests

from langchain.tools import tool

def get_osv_data(cve_id: str) -> DebianCVE | str:
    try:
        response = requests.get(f"https://api.osv.dev/v1/vulns/DEBIAN-{cve_id}")
        response.raise_for_status()
        data = response.json()
        debian_cve = DebianCVE(**data)
        filtered = debian_cve.filter_for_current_system()
        return filtered
    except requests.RequestException as e:
        return f"Error fetching OSV data for {cve_id}: {e}"

@tool
def scan_system_status(top: int = 5) -> str:
    """Scans the system for vulnerabilities and returns summary for `top` vulnerabilities
    """
    try:
        result = subprocess.run(['debsecan'], 
                                capture_output=True,
                                text=True, 
                                check=True).stdout
        vulns = {v for line in result.splitlines() for v in line.split()}
        vulnerabilities = []
        for cve in vulns:
            epss, percentile = epss_lookup.get(cve, (0.0, 0.0))
            vulnerabilities.append(
                Vulnerability(cve=cve, epss=epss, percentile=percentile)
            )
        vulnerabilities.sort(key=lambda v: v.epss, reverse=True)
        top_vulns = vulnerabilities[:top]
        debian_data = [get_osv_data(vuln.cve) for vuln in top_vulns]
        if not all(isinstance(data, DebianCVE) for data in debian_data):
            return f"TOOL ERROR: Failed to retrieve complete OSV data. {debian_data}"
        
        return "\n\n".join([data.to_llm_summary(*epss_lookup.get(data.id.split('DEBIAN-')[-1])) for data in debian_data])
    except Exception as e:
        return f"TOOL ERROR: Failed get system status: {e}"


In [50]:
print(scan_system_status())

CVE_ID: DEBIAN-CVE-2023-44487
CRITICALITY: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H
DETAILS: The HTTP/2 protocol allows a denial of service (server resource consumption) because request cancellation can reset many streams quickly, as exploited in the wild in August through October 2023....
PACKAGE: dnsdist | STATUS: FIX_AVAILABLE | Fixed in: 1.8.2-2
PACKAGE: grpc | STATUS: AWAITING_FIX | No fix version assigned yet.
PACKAGE: haproxy | STATUS: FIX_AVAILABLE | Fixed in: 1.8.13-1
PACKAGE: jetty9 | STATUS: FIX_AVAILABLE | Fixed in: 9.4.53-1
PACKAGE: netty | STATUS: FIX_AVAILABLE | Fixed in: 1:4.1.48-8
PACKAGE: nghttp2 | STATUS: FIX_AVAILABLE | Fixed in: 1.57.0-1
PACKAGE: nginx | STATUS: FIX_AVAILABLE | Fixed in: 1.24.0-2
PACKAGE: tomcat10 | STATUS: FIX_AVAILABLE | Fixed in: 10.1.14-1
PACKAGE: tomcat9 | STATUS: FIX_AVAILABLE | Fixed in: 9.0.70-2
PACKAGE: varnish | STATUS: FIX_AVAILABLE | Fixed in: 7.5.0-1
EPSS_SCORE: 0.94466 | PERCENTILE: 0.99995

CVE_ID: DEBIAN-CVE-2024-1086
CRITICALIT

In [57]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent

llm = ChatOllama(
    model="llama3.2",
    base_url="http://localhost:11434",
    temperature=0
)

tools = [scan_system_status]

prompt = """
You are a Senior Debian Security Analyst with expertise in vulnerability assessment and risk management. Your role is to analyze system vulnerabilities and provide actionable security recommendations.

## Core Responsibilities
- Analyze system vulnerability data with precision and accuracy
- Prioritize threats based on EPSS scores and system impact
- Provide clear, actionable security recommendations
- Maintain data integrity and evidence-based analysis

## Operational Guidelines

### Tool Usage
- **ALWAYS** use the `scan_system_status` tool when users request vulnerability information
- Default to analyzing the top 6 vulnerabilities unless a specific number is requested
- Process all tool responses completely before providing recommendations

### Analysis Requirements
1. **Vulnerability Assessment**: Analyze each CVE's EPSS score, percentile ranking, and potential system impact
2. **Risk Prioritization**: Rank vulnerabilities by exploitability probability and potential damage
3. **CVSS Integration**: When multiple CVSS scores are present, calculate a weighted average or identify the most relevant score for the current system context

### Output Standards
- **Format**: Always present findings in well-structured markdown with clear headings and bullet points
- **Evidence-Based**: Base ALL recommendations solely on data returned from tools - never fabricate or assume information
- **Actionable**: Include specific remediation steps, patch priorities, and mitigation strategies
- **Professional**: Use clear, technical language appropriate for security professionals

### Response Structure
1. **Executive Summary**: Brief overview of security posture
2. **Critical Vulnerabilities**: Top threats requiring immediate attention
3. **Risk Analysis**: EPSS scores, exploitability assessment, and impact evaluation
4. **Recommendations**: Prioritized action items with specific remediation steps
5. **Additional Context**: Relevant security considerations and monitoring suggestions

Remember: Your analysis directly impacts system security decisions. Maintain accuracy, clarity, and actionable insight in all responses.
"""

agent = create_agent(model=llm,
                        tools=tools,
                        system_prompt=prompt)

# The input format remains the same as your .invoke() call
inputs = {
    "messages": [
        {"role": "user", "content": "Scan my system and tell me the top 10 vulnerabilities."}
    ]
}


# Iterate through the stream
for chunk in agent.stream(inputs, stream_mode="updates"):
    for step_name, data in chunk.items():
        print(f"\n[Step: {step_name}]")
        
        # Pull the last message from this step's update
        last_msg = data["messages"][-1]
        
        # Use pretty_print() for a clean display of each step
        last_msg.pretty_print()

        # Access token usage metadata if it exists
        if hasattr(last_msg, 'usage_metadata') and last_msg.usage_metadata:
            usage = last_msg.usage_metadata
            print(f"\n--- Token Stats ---")
            print(f"Input Tokens: {usage.get('input_tokens')}")
            print(f"Output Tokens: {usage.get('output_tokens')}")
            print(f"Total Tokens: {usage.get('total_tokens')}")        


[Step: model]
================================== Ai Message ==================================
Tool Calls:
  scan_system_status (22953b95-2dba-4df9-9eae-f24f5dfc9a80)
 Call ID: 22953b95-2dba-4df9-9eae-f24f5dfc9a80
  Args:
    top: 10

--- Token Stats ---
Input Tokens: 541
Output Tokens: 18
Total Tokens: 559

[Step: tools]
================================= Tool Message =================================
Name: scan_system_status

CVE_ID: DEBIAN-CVE-2023-44487
CRITICALITY: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H
DETAILS: The HTTP/2 protocol allows a denial of service (server resource consumption) because request cancellation can reset many streams quickly, as exploited in the wild in August through October 2023....
PACKAGE: dnsdist | STATUS: FIX_AVAILABLE | Fixed in: 1.8.2-2
PACKAGE: grpc | STATUS: AWAITING_FIX | No fix version assigned yet.
PACKAGE: haproxy | STATUS: FIX_AVAILABLE | Fixed in: 1.8.13-1
PACKAGE: jetty9 | STATUS: FIX_AVAILABLE | Fixed in: 9.4.53-1
PACKAGE: netty | STAT

## Llama 3.1 output

**Executive Summary**

This report provides a summary of the top 10 vulnerabilities found on your system, along with their corresponding Criticality scores and potential impact.

**Critical Vulnerabilities**

1. **DEBIAN-CVE-2023-44487**: HTTP/2 protocol vulnerability allowing denial-of-service (server resource consumption)
	* EPSS Score: 0.94466
	* Percentile Ranking: 0.99995
	* Recommended Action: Update dnsdist package to version 1.8.2-2
2. **DEBIAN-CVE-2024-1086**: Linux kernel vulnerability allowing local privilege escalation
	* EPSS Score: 0.85317
	* Percentile Ranking: 0.9933
	* Recommended Action: Update linux package to version 6.6.15-1
3. **DEBIAN-CVE-2023-0386**: Linux kernel vulnerability allowing unauthorized access to execution of setuid files with capabilities
	* EPSS Score: 0.5784
	* Percentile Ranking: 0.98101
	* Recommended Action: Update linux package to version 6.1.11-1

**Risk Analysis**

| CVE ID | Criticality | EPSS Score | Percentile Ranking |
| --- | --- | --- | --- |
| DEBIAN-CVE-2023-44487 | CRITICAL | 0.94466 | 0.99995 |
| DEBIAN-CVE-2024-1086 | CRITICAL | 0.85317 | 0.9933 |
| DEBIAN-CVE-2023-0386 | HIGH | 0.5784 | 0.98101 |

**Recommendations**

1. **Update dnsdist package**: Update the dnsdist package to version 1.8.2-2 to fix the HTTP/2 protocol vulnerability.
2. **Update linux package**: Update the linux package to version 6.6.15-1 to fix the Linux kernel vulnerability allowing local privilege escalation.
3. **Update linux package**: Update the linux package to version 6.1.11-1 to fix the Linux kernel vulnerability allowing unauthorized access to execution of setuid files with capabilities.

**Additional Context**

Please note that these vulnerabilities are critical and require immediate attention. It is recommended to apply the necessary patches and updates as soon as possible to prevent potential security breaches.

Monitoring suggestions:

* Regularly check for updates on the above-mentioned packages.
* Monitor system logs for any suspicious activity related to these vulnerabilities.
* Implement additional security measures, such as intrusion detection systems and network monitoring tools, to detect and respond to potential threats.

## Llama3.2 Output

**Executive Summary**

This report provides a summary of the top 10 vulnerabilities found on your system, along with their corresponding Criticality scores and potential impact.

**Critical Vulnerabilities**

1. **DEBIAN-CVE-2023-44487**: HTTP/2 protocol vulnerability allowing denial-of-service (server resource consumption)
	* EPSS Score: 0.94466
	* Percentile Ranking: 0.99995
	* Recommended Action: Update dnsdist package to version 1.8.2-2
2. **DEBIAN-CVE-2024-1086**: Linux kernel vulnerability allowing local privilege escalation
	* EPSS Score: 0.85317
	* Percentile Ranking: 0.9933
	* Recommended Action: Update linux package to version 6.6.15-1
3. **DEBIAN-CVE-2023-0386**: Linux kernel vulnerability allowing unauthorized access to execution of setuid files with capabilities
	* EPSS Score: 0.5784
	* Percentile Ranking: 0.98101
	* Recommended Action: Update linux package to version 6.1.11-1

**Risk Analysis**

| CVE ID | Criticality | EPSS Score | Percentile Ranking |
| --- | --- | --- | --- |
| DEBIAN-CVE-2023-44487 | CRITICAL | 0.94466 | 0.99995 |
| DEBIAN-CVE-2024-1086 | CRITICAL | 0.85317 | 0.9933 |
| DEBIAN-CVE-2023-0386 | HIGH | 0.5784 | 0.98101 |

**Recommendations**

1. **Update dnsdist package**: Update the dnsdist package to version 1.8.2-2 to fix the HTTP/2 protocol vulnerability.
2. **Update linux package**: Update the linux package to version 6.6.15-1 to fix the Linux kernel vulnerability allowing local privilege escalation.
3. **Update linux package**: Update the linux package to version 6.1.11-1 to fix the Linux kernel vulnerability allowing unauthorized access to execution of setuid files with capabilities.

**Additional Context**

Please note that these vulnerabilities are critical and require immediate attention. It is recommended to apply the necessary patches and updates as soon as possible to prevent potential security breaches.

Monitoring suggestions:

* Regularly check for updates on the above-mentioned packages.
* Monitor system logs for any suspicious activity related to these vulnerabilities.
* Implement additional security measures, such as intrusion detection systems and network monitoring tools, to detect and respond to potential threats.


In [53]:
print(f"Total messages in history: {len(inputs['messages'])}")
for msg in inputs['messages']:
    print(msg)

Total messages in history: 1
{'role': 'user', 'content': 'Scan my system and tell me the top 3 vulnerabilities.'}
